In [6]:
from numpy import char
from datetime import date
import pandas as pd
import numpy as np
import time
pd.set_option('display.float_format', str)

In [7]:
l_columnnames = ["L_ORDERKEY", "L_PARTKEY", "L_SUPPKEY", "L_LINENUMBER", "L_QUANTITY", "L_EXTENDEDPRICE", "L_DISCOUNT", "L_TAX", "L_RETURNFLAG", "L_LINESTATUS", "L_SHIPDATE", "L_COMMITDATE", "L_RECEIPTDATE", "L_SHIPINSTRUCT","L_SHIPMODE", "L_COMMENT"]

for i in range(len(l_columnnames)):
    l_columnnames[i] = l_columnnames[i].lower()
    
l_data_types = {
    'l_orderkey': int,
    'l_partkey': int,
    'l_suppkey': int,
    'l_linenumber': int,
    'l_quantity': float,
    'l_extendedprice': float,
    'l_discount': float,
    'l_tax': float,
    'l_returnflag': str,
    'l_linestatus': str,
    'l_shipinstruct': str,
    'l_shipmode': str,
    'l_comment': str
}

l_parse_dates = ['l_shipdate', 'l_commitdate', 'l_receiptdate']

In [8]:
# Don't set indexes, as we can't access them with Pandas selection!
lineitem = pd.read_table("../tpch-pgsql-master/data/load/lineitem.tbl.csv", sep="|", names=l_columnnames, dtype=l_data_types, parse_dates=l_parse_dates)

In [9]:
p_columnnames = ["P_PARTKEY", "P_NAME", "P_MFGR", "P_BRAND", "P_TYPE", "P_SIZE", "P_CONTAINER", "P_RETAILPRICE", "P_COMMENT"]

for i in range(len(p_columnnames)):
    p_columnnames[i] = p_columnnames[i].lower()
    
p_data_types = {
    'p_partkey': int, 
    'p_name': str,
    'p_mfgr': str,
    'p_brand': str,
    'p_type': str,
    'p_size': int,
    'p_container': str,
    'p_retailprice': float,
    'p_comment': str
}

p_parse_dates = []

In [10]:
# Don't set indexes, as we can't access them with Pandas selection!
part = pd.read_table("../tpch-pgsql-master/data/load/part.tbl.csv", sep="|", names=p_columnnames, dtype=p_data_types, parse_dates=p_parse_dates)

## Q14

In [11]:
start_t = time.time()

df_filter_1 = lineitem[(lineitem.l_shipdate >= pd.Timestamp('1995-09-01 00:00:00')) & (lineitem.l_shipdate < pd.Timestamp('1995-10-01 00:00:00'))]
df_filter_1 = df_filter_1[['l_extendedprice', 'l_discount', 'l_partkey']]
df_filter_2 = part[['p_type', 'p_partkey']]
df_merge_1 = df_filter_1.merge(df_filter_2, left_on="l_partkey", right_on="p_partkey")
df_merge_1 = df_merge_1[['p_type', 'l_extendedprice', 'l_discount']]
df_aggr_1 = pd.DataFrame()
df_aggr_1['promo_revenue'] = [100.00 * (df_merge_1.apply(lambda x: ( x["l_extendedprice"] * ( 1 - x["l_discount"] )) if x["p_type"].startswith("PROMO") else 0, axis=1)).sum() / (df_merge_1.l_extendedprice * ( 1 - df_merge_1.l_discount )).sum()]
df_aggr_1 = df_aggr_1[['promo_revenue']]
df_limit_1 = df_aggr_1[['promo_revenue']]
result = df_limit_1.head(1)

print(len(result))
print(result)

print("---" * 5)
print(time.time() - start_t)
print("---" * 5)

1
       promo_revenue
0 16.380778626395536
---------------
0.9248039722442627
---------------
